# Crawler no Linkedin
### Este projeto faz a captura diversas informações dos seus seguidores. 

In [1]:
# Instalar a biblioteca tqdm 
# Cria a barra de progresso
!pip install tqdm

In [2]:
# Instalar a biblioteca pyppeteer
# Necessária para executar o browser
!pip install pyppeteer

In [3]:
# Importação das bibliotecas necessárias
import asyncio
from pyppeteer import launch
import pandas as pd
import requests
from json import loads
import time
import random
from datetime import datetime
from tqdm.notebook import tqdm

# Inicialização do browser para o usuário logar

Faça login até chegar na página inicial do seu linkedin


In [4]:
browser = await launch(
        headless=False,
        args=['--no-sandbox'],
        autoClose=False
    )
page = await browser.newPage()
await page.goto("https://www.linkedin.com/")

# digite se login e senha no browser e siga até estar na sua página inicial

# Vai até a página de seus seguidores para pegar headers da API

In [5]:
await page.goto("https://www.linkedin.com/feed/followers/")
global headers
headers = {}
await page.setRequestInterception(True)
async def intercept(request):
    if ("https://www.linkedin.com/voyager/api/feed/richRecommendedEntities" in request.url ):
        global headers
        headers = request.headers
        print(len(headers))
    await request.continue_()
page.on('request', lambda req: asyncio.ensure_future(intercept(req)))
await page.evaluate( "() => { window.scrollBy(0, window.innerHeight); }");
time.sleep(5)
cookie1 = await page.cookies()
s = []
for e in cookie1:
    s.append(f"{e['name']}={e['value']}")
cookie1_str = "; ".join(s)
headers = {key: str(value) for key, value in headers.items()}
headers["cookie"] = cookie1_str

8


# Funções para puxar dados do LinkedIn

In [6]:
# Função para capturar os dados da página do Linkedin
def get_data(countw, index_start, url, headers):
    try:
        resp = requests.get(url=url.format(countw=countw, index_start=index_start), headers=headers)
        if resp.status_code == 200:
            data = loads(resp.content.decode("utf-8"))
            return data
        else:
            raise Exception(f"not allowed. headers or coockies problems. Status: {resp.status_code}")
    except Exception as e:
        print(f"error: {e}")
        return []
    
def agrupa_entidades(elements, data):
    for element in data['data']['elements']:
        for inc in data['included']:
            ek = element.get("*recommendedEntity", "a").split(":member:")[-1]
            ik = inc.get('entityUrn',"b").split(":")[-1]
            if ek == ik:
                e = dict()
                e.update(element)
                e.update(inc)
                elements[ek] = elements.get(ek, {})
                elements[ek].update(e)
    return elements


# Cria o dicionario com as colunas necessárias para armazenar os dados do Linkedin
def dict_element(e):
    return {
        "seguidores": e["followerCount"],
        "influenciador": e["influencer"],
        "nome": f"{e['firstName']} {e['lastName']}",
        "cargo": e["occupation"],
        "link": f'https://www.linkedin.com/in/{e["publicIdentifier"]}',
        "prova_social": e["socialProofTotalCount"],
        "publicacoes": f'https://www.linkedin.com/in/{e["publicIdentifier"]}/detail/recent-activity/shares/',
        "localizacao": e["locationName"]
    }

# Armazena os dados em um dataframe
def dataFrame_element(e):
    return pd.DataFrame.from_dict([dict_element(e)])


# Função para recuperar os dados da página do Linkedin
def get_data_in_range(groups_qty, countw, first_index_start, url_inf_scroll, headers, wait_time, wait_random_time):
    df = pd.DataFrame([], columns=["seguidores", "influenciador", "nome", "cargo", "link", "prova_social", "publicacoes", "localizacao"])
    for i in tqdm(range(groups_qty)):
        # data1 = get_data(countw, 1000 + countw*i, url_inf_scroll, headers)
        data1 = get_data(countw, countw*i, url_inf_scroll, headers)
        elements1 = dict()
        agrupa_entidades(elements1, data1)
        time.sleep(wait_time + wait_random_time * random.random())
        for e in elements1.values():
            df_t = dataFrame_element(e)
            df = df.append(df_t, ignore_index=True)
    return df
            

# Essa etapa executa de fato a consulta

No primeiro parâmetro você passa seu número de seguidores divido por 20. Pode passar menos também, para trazer apenas uma amostra.
Os dois ultimos parâmetros são de tempo para esperar entre uma requisição. Se quiser zerar ele fica mais rápido, mas não recomendo.

In [7]:
countw = 20
index_start = 280
url_inf_scroll = "https://www.linkedin.com/voyager/api/feed/richRecommendedEntities?count={countw}&q=followerRecommendations&start={index_start}"

df = get_data_in_range(124, countw, 0, url_inf_scroll, headers, .1, .5)
df

,seguidores,influenciador,nome,cargo,link,prova_social,publicacoes,localizacao
0,17028,False,Prof. Gretz,PALESTRANTE PENTA CAMPEÃO DO TOP OF MIND,https://www.linkedin.com/in/profgretz,3,https://www.linkedin.com/in/profgretz/detail/r...,None
1,1048,False,Joab Santana Santos,Biologist - Data Scientist || Python || Machin...,https://www.linkedin.com/in/joabsantanna,0,https://www.linkedin.com/in/joabsantanna/detai...,None
2,752,False,"Amazile López, PhD",Data Analytics | Análise de Dados | Data Scien...,https://www.linkedin.com/in/amazilelópez,0,https://www.linkedin.com/in/amazilelópez/detai...,None
3,12269,False,Anderson Carlos da Silva Pedro,Pós-Graduação Latu Sensu – Especialização em G...,https://www.linkedin.com/in/anderson-carlos-da...,4,https://www.linkedin.com/in/anderson-carlos-da...,None
4,443,False,João Gross,Master Researcher at Federal University of Rio...,https://www.linkedin.com/in/jlggross,0,https://www.linkedin.com/in/jlggross/detail/re...,None
...,...,...,...,...,...,...,...,...
1461,432,False,José Albuquerque Moreira,Professor na www moodlemais moodlecloud com,https://www.linkedin.com/in/josé-albuquerque-m...,0,https://www.linkedin.com/in/josé-albuquerque-m...,None
1462,953,False,Wilson dos Anjos Junior,Data Scientist at Beegol,https://www.linkedin.com/in/wilson-dos-anjos-j...,1,https://www.linkedin.com/in/wilson-dos-anjos-j...,None
1463,1664,False,Karinne Cristina,Cientista de Dados Júnior | Refinaria de Dados,https://www.linkedin.com/in/karinnecristinaper...,4,https://www.linkedin.com/in/karinnecristinaper...,None
1464,6342,False,Alaap Dhall,Result Oriented Data Scientist | #ONO | Kaggle...,https://www.linkedin.com/in/alaapdhall,2,https://www.linkedin.com/in/alaapdhall/detail/...,None


#  Exibe o data frame ordenado por quantidade de seguidores

In [8]:
df.sort_values(by="seguidores", ascending=False).head()


,seguidores,influenciador,nome,cargo,link,prova_social,publicacoes,localizacao
28,39237,False,Diego Cidade,CEO da Academia do Universitário | LinkedIn To...,https://www.linkedin.com/in/diegocidade,12,https://www.linkedin.com/in/diegocidade/detail...,None
177,35695,False,Marco Tavora Ph.D.,Theoretical Physicist | Science and Mathematic...,https://www.linkedin.com/in/marco-tavora,14,https://www.linkedin.com/in/marco-tavora/detai...,None
1207,34098,False,Ana Mazzi,Talent Acquisition | Recruiter | Recrutamento ...,https://www.linkedin.com/in/ana-mazzi,14,https://www.linkedin.com/in/ana-mazzi/detail/r...,None
1432,32714,False,Ilya Brotzky,CEO & Founder VanHack | Techstars | G20 Delega...,https://www.linkedin.com/in/ilyabrotzky,10,https://www.linkedin.com/in/ilyabrotzky/detail...,None
509,31341,False,Elisete Vasconcelos _ Perfil Cheio,Talent Acquisition_Recursos Humanos _ Tech Rec...,https://www.linkedin.com/in/elisete-vasconcelo...,15,https://www.linkedin.com/in/elisete-vasconcelo...,None


# Salva o dataframe em csv com a data e hora

In [9]:
df.sort_values(by="seguidores", ascending=False).to_csv(f'crawledin-{datetime.now().strftime("%Y-%m-%d_%H-%M")}.csv', index=False)

# Visita a página de seguidores com conteudo.

In [ ]:
#
#for index, row in df.sample(20).iterrows():
#    print(row['link'])
#    await page.goto(row['publicacoes'])
#    elements = (await page.querySelectorAll("h1"))
#    tem_post = True
#    for e in elements:
#        if "Nada para ver por enquanto" in await page.evaluate('(element) => element.textContent', e):
#            tem_post = False
#    if tem_post:
#        time.sleep(9 + 2 * random.random())
#    else:
#        time.sleep(1)

